In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from collections import Counter
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingClassifier
import seaborn as sns

In [2]:
data=sns.load_dataset('titanic')

In [3]:
data

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [4]:
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [5]:
data.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [6]:
data["age"]=data["age"].ffill()
data["deck"]=data["deck"].bfill()
data['embarked']=data["embarked"].ffill()
data["embark_town"]=data["embark_town"].ffill()

In [15]:
data.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           0
embark_town    0
alive          0
alone          0
dtype: int64

In [14]:
data["deck"]=data["deck"].ffill()

In [16]:
l=LabelEncoder()
data["alone"]=l.fit_transform(data["alone"])

In [17]:
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,C,Southampton,no,0
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,0
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,C,Southampton,yes,1
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,0
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,E,Southampton,no,1


In [18]:
data["alone"].unique()

array([0, 1])

In [19]:
data["alone"].value_counts()

1    537
0    354
Name: alone, dtype: int64

# Q1 Find the females those who are not alone with fare values is less than 60,000 & greater than 20,000

In [20]:
a=data.loc[((data["sex"]=="female")&(data["alone"]==0)&((data["fare"]<=60)&(data["fare"]>=20)))]

In [21]:
a

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,0
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,G,Cherbourg,yes,0
24,0,3,female,8.0,3,1,21.0750,S,Third,child,False,C,Southampton,no,0
25,1,3,female,38.0,1,5,31.3875,S,Third,woman,False,C,Southampton,yes,0
41,0,2,female,27.0,1,0,21.0000,S,Second,woman,False,D,Southampton,no,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,1,1,female,47.0,1,1,52.5542,S,First,woman,False,D,Southampton,yes,0
874,1,2,female,28.0,1,0,24.0000,C,Second,woman,False,C,Cherbourg,yes,0
880,1,2,female,25.0,0,1,26.0000,S,Second,woman,False,B,Southampton,yes,0
885,0,3,female,39.0,0,5,29.1250,Q,Third,woman,False,B,Queenstown,no,0


In [22]:
a["alone"].unique()

array([0])

# Find the third class travelled people and find the count of teenage & adult_age &Old_age

In [23]:
data["class"].unique()

['Third', 'First', 'Second']
Categories (3, object): ['First', 'Second', 'Third']

In [24]:
AgeCat=[]
for x in data["age"]:
    if x<=15:
        AgeCat.append("child")
    elif x>15 and x<=30:
        AgeCat.append("teenage")
    elif x>30 and x<=50:
        AgeCat.append("adult")
    else:
        AgeCat.append("oldage")

In [25]:
data.insert(5,"AgeCat",AgeCat)

In [26]:
data["AgeCat"].value_counts()

teenage    401
adult      307
child      106
oldage      77
Name: AgeCat, dtype: int64

In [27]:
data.head()

,survived,pclass,sex,age,sibsp,AgeCat,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,teenage,0,7.2500,S,Third,man,True,C,Southampton,no,0
1,1,1,female,38.0,1,adult,0,71.2833,C,First,woman,False,C,Cherbourg,yes,0
2,1,3,female,26.0,0,teenage,0,7.9250,S,Third,woman,False,C,Southampton,yes,1
3,1,1,female,35.0,1,adult,0,53.1000,S,First,woman,False,C,Southampton,yes,0
4,0,3,male,35.0,0,adult,0,8.0500,S,Third,man,True,E,Southampton,no,1


In [28]:
b=data.loc[(data["class"]=="Third")&(data["AgeCat"]!="child")]

In [29]:
b

,survived,pclass,sex,age,sibsp,AgeCat,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,teenage,0,7.2500,S,Third,man,True,C,Southampton,no,0
2,1,3,female,26.0,0,teenage,0,7.9250,S,Third,woman,False,C,Southampton,yes,1
4,0,3,male,35.0,0,adult,0,8.0500,S,Third,man,True,E,Southampton,no,1
5,0,3,male,35.0,0,adult,0,8.4583,Q,Third,man,True,E,Queenstown,no,1
8,1,3,female,27.0,0,teenage,2,11.1333,S,Third,woman,False,G,Southampton,yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,0,3,female,22.0,0,teenage,0,10.5167,S,Third,woman,False,B,Southampton,no,1
884,0,3,male,25.0,0,teenage,0,7.0500,S,Third,man,True,B,Southampton,no,1
885,0,3,female,39.0,0,adult,5,29.1250,Q,Third,woman,False,B,Queenstown,no,0
888,0,3,female,19.0,1,teenage,2,23.4500,S,Third,woman,False,C,Southampton,no,0


In [30]:
b["AgeCat"].value_counts()

teenage    257
adult      142
oldage      20
Name: AgeCat, dtype: int64

# Find the count of teenage, Adult_age,& Old_age

In [36]:
c=data.loc[((data["AgeCat"]=="teenage")|(data["AgeCat"]=="adult")|(data["AgeCat"]=="oldage"))]

In [37]:
c

,survived,pclass,sex,age,sibsp,AgeCat,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,teenage,0,7.2500,S,Third,man,True,C,Southampton,no,0
1,1,1,female,38.0,1,adult,0,71.2833,C,First,woman,False,C,Cherbourg,yes,0
2,1,3,female,26.0,0,teenage,0,7.9250,S,Third,woman,False,C,Southampton,yes,1
3,1,1,female,35.0,1,adult,0,53.1000,S,First,woman,False,C,Southampton,yes,0
4,0,3,male,35.0,0,adult,0,8.0500,S,Third,man,True,E,Southampton,no,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,teenage,0,13.0000,S,Second,man,True,B,Southampton,no,1
887,1,1,female,19.0,0,teenage,0,30.0000,S,First,woman,False,B,Southampton,yes,1
888,0,3,female,19.0,1,teenage,2,23.4500,S,Third,woman,False,C,Southampton,no,0
889,1,1,male,26.0,0,teenage,0,30.0000,C,First,man,True,C,Cherbourg,yes,1


In [38]:
c["AgeCat"].value_counts()

teenage    401
adult      307
oldage      77
Name: AgeCat, dtype: int64

# Find Alive man & childs from 1st & 2nd class with fare of above 25000

In [39]:
d=data.loc[(data["alive"]=="yes")&((data["who"]=="man")|(data["who"]=="child"))&(data["fare"]>=25)]

In [40]:
d

,survived,pclass,sex,age,sibsp,AgeCat,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
9,1,2,female,14.0,1,child,0,30.0708,C,Second,child,False,G,Cherbourg,yes,0
23,1,1,male,28.0,0,teenage,0,35.5000,S,First,man,True,A,Southampton,yes,1
43,1,2,female,3.0,1,child,2,41.5792,C,Second,child,False,D,Cherbourg,yes,0
55,1,1,male,65.0,0,oldage,0,35.5000,S,First,man,True,C,Southampton,yes,1
58,1,2,female,5.0,1,child,2,27.7500,S,Second,child,False,B,Southampton,yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,1,2,male,1.0,0,child,2,37.0042,C,Second,child,False,B,Cherbourg,yes,0
838,1,3,male,32.0,0,adult,0,56.4958,S,Third,man,True,C,Southampton,yes,1
839,1,1,male,32.0,0,adult,0,29.7000,C,First,man,True,C,Cherbourg,yes,1
857,1,1,male,51.0,0,oldage,0,26.5500,S,First,man,True,E,Southampton,yes,1
